In [ ]:
# Import project dependencies

# Pandas
import pandas as pd

# Numpy
import numpy as np

#sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score,
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn.manifold import TSNE 

# NLTK
from nltk.corpus import stopwords
from nltk import  word_tokenize

# Keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant
from keras import backend as K
from keras.layers import MaxPool1D
from keras.models import Model
from keras.layers.core import *
from keras.models import *
from keras.layers import LSTM, concatenate,Bidirectional,Multiply

#Seq2Seq
from keras_self_attention import SeqSelfAttention

# Tensorflow
from tensorflow.keras.layers import Attention

# Matplotlib
from matplotlib import pyplot
import matplotlib.pyplot as plt

# Other dependencies
from scipy import spatial
import io
import os
import re
from collections import Counter
from wordcloud import WordCloud
import seaborn as sns

In [192]:
# Read csv dataset and save to df

filePath = 'TOTAL_TEXTS_SETOPATI.csv'
df = pd.read_csv(filePath)

# Define Constant Variables
VALIDATION_SPLIT = 0.1
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 300

# computing number of rows
rows = len(df.axes[0])
 
# computing number of columns
cols = len(df.axes[1])
print("DataFrame Shape: "+ str(df.shape))
print("Row Size: "+ str(rows))
print("Column Size: "+ str(cols))

DataFrame Shape: (104047, 5)
Row Size: 104047
Column Size: 5


In [193]:
#Preprocessing {Remove Stop words using nltk}

stop_words = stopwords.words("nepali")
def string_manipulation(df,column)  : 
    df[column] = df[column].apply(remove_special_characters)
    df[column] = df[column].apply(remove_non_useful_characters)
    df[column] = df[column].apply(lambda x: " ".join([i for i in x.split()if i not in (stop_words)]))
    return df

def remove_special_characters(input_string):
    return re.sub('[।(),<<?!,—,–,/,’,‘,:,\u200d]', '', input_string)

def remove_non_useful_characters(input_string):
    return re.sub(r'[^\u0900-\u097F\s]', '', input_string)

print("Before preprocessing\n")
print(df["text"][1])
Data = string_manipulation(df,"text")
print("After preprocessing\n")
print(Data["text"][1])

Before preprocessing

\n\tकाठमाडौंको कीर्तिपुर क्रिकेट मैदानमा भोलि हुने नेपाल र नामिवियाबीचको खेलमा नेपाल प्रहरीले सुरक्षाका लागि प्रहरी उपरीक्षकको कमाण्डमा ७ सय ७३ जना प्रहरी परिचालन गर्ने भएको छ।\n\n\t\xa0\n\n\tमहानगरीय प्रहरी कार्यालय रानीपोखरीले शुक्रबार पत्रकार सम्मेलन गरेर काठमाडौं प्रहरी परिसरका एसपी प्रद्युम्न कार्कीको कमाण्डमा ७ सय ७३ जना प्रहरी परिचालन गर्ने एआईजी प्रतापसिंह थापाले जानकारी दिए।\xa0\n\n\t\xa0\n\n\tथापाका अनुसार सादा पोशाकमा पनि प्रहरी परिचालन गरिनेछ। नेपाल प्रहरीको साथै खेल मैदानमा सशस्त्र प्रहरी बलका सय भन्दा बढी प्रहरी पनि परिचालन हुने भएको छ।थप आवश्यक परे सशस्त्र प्रहरी अरु पनि खटाउन सकिने सशस्त्र प्रहरीका प्रवक्ता डिआइजी शैलेन्द्र खनालले बताए। क्रिकेट मैदानसँगै भित्र पट्टि नेपाल प्रहरीको एक घेरा हुनेछ भने त्यसपछि दर्शकमाझ र दर्शकबाहिर गरी ३ घेरा राखिने प्रहरीले बताएको छ। \xa0\n\n\t\xa0\n\n\tनेपाल प्रहरीको घेराबाहिर सशस्त्र प्रहरीले पनि मैदानलाई सुरक्षा दिने प्रहरीले बताएको छ। अन्तर्राष्ट्रियस्तरको खेल भएकाले सुरक्षा पनि अन्तर्राष्ट्रियस्तरकै हुने एआईजी था

In [194]:
# convert word into vector

print('Indexing word vectors.')
embeddings_dict = {}

# Used NPVec1: Word Embeddings for Nepali for glove from [https://github.com/nowalab/nepali-word-embeddings]
# processed.glove.txt, processed_normalized_stemmed.glove 

with open("processed.glove", 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        token = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[token] = vector

print('Found %s word vectors.' % len(embeddings_dict))

Indexing word vectors.
Found 663564 word vectors.


In [196]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [197]:
# Prepare text samples and their labels

newsTexts = Data['text'].tolist()
labels_map = {}  # dictionary mapping label name to numeric id
labels = [] # list of label ids
allCategoryDataList = Data['catagory']

indexCount = 0
for category in allCategoryDataList.unique():
    labels_map[category] = indexCount
    indexCount = indexCount + 1

for category in allCategoryDataList.tolist():
    labels.append(labels_map[category])
        
print('Found %s news texts.' % len(newsTexts))
print('Categories with label ids:\n %s' % labels_map)
display(df.loc[0:4])



Found 104047 news texts.
Categories with label ids:
 {'new-news': 0, 'archive-news': 1, 'politics': 2, 'sports': 3, 'sports/anya-khelkud': 4, 'sports/cricket': 5, 'sports/football': 6, 'sports/noc': 7, 'sports/sports-activity': 8, 'sports/volleyball': 9, 'art': 10, 'art/art-activity': 11, 'blog': 12, 'ghumphir/ghumphir-activity': 13, 'ghumphir/ghumphir-experience': 14, 'ghumphir/ghumphir-news': 15, 'global': 16, 'global/america': 17, 'global/asia': 18, 'global/australia': 19, 'global/belgium': 20, 'global/canada': 21, 'global/china': 22, 'global/denmark': 23, 'global/france': 24, 'global/india': 25, 'global/japan': 26, 'global/jeneva': 27, 'global/korea': 28, 'global/malaysia': 29, 'global/norway': 30, 'global/other-global': 31, 'global/portugal': 32, 'global/qatar': 33, 'global/saudi-arabia': 34, 'global/uae': 35, 'global/uk': 36, 'kinmel': 37, 'kinmel/agriculture': 38, 'kinmel/automobiles': 39, 'kinmel/banking': 40, 'kinmel/business': 41, 'kinmel/construction': 42, 'kinmel/employment

,catagory,heading,text,date,link
0,new-news,मेसीलाई तुरुन्त अनुबन्धित गर्न बार्सिलोनालाई न...,ब्राजिलका स्टार स्ट्राइकर नेइमारले अर्जेन्टिनी...,"बुधबार, वैशाख २७, २०७४, ००:४२:२९",https://dev.setoparty.com/new-news/45151
1,archive-news,नेपाल र नामिबियाबीचको खेलमा यस्तो छ सुरक्षा,काठमाडौंको कीर्तिपुर क्रिकेट मैदानमा भोलि नेपा...,2016-04-15 16:41:04,https://dev.setoparty.com/archive-news/45151
2,politics,अलबिदा मणी शाह (भिडियो र फोटोफिचर),नेपाली राष्ट्रिय फुटबल टोलीका खेलाडी मणी विक्र...,"मंगलबार, जेठ १, २०७५, ०३:०५:४२",https://dev.setoparty.com/politics/158282
3,politics,मलेसियालाई १० विकेटले हराउँदै नेपाल एसिया कपमा...,मलेसियालाई १० विकेटको फराकिलो अन्तरमा हराउँदै ...,"शनिबार, भदौ ३१, २०७४, ०३:२२:४७",https://dev.setoparty.com/politics/87233
4,sports,अर्धसतक पूरा गर्दै अाउट भए ज्ञानेन्द्र,विश्व क्रिकेट लिग डब्लुसिएल च्याम्पियनसिप नेपा...,"बुधबार, मंसिर २०, २०७४, ०२:११:१६",https://dev.setoparty.com/sports/111454


In [198]:
# Prepare word tokens to numeric value
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(newsTexts)
word_index = tokenizer.word_index

print('Found %s unique tokens.' % len(word_index))

def printWordIndex(limit): 
    counter = 0
    for word, index in tokenizer.word_index.items():
        if counter < limit:
            print(f"{word}: {index}")
            counter += 1
        else:
            break

printWordIndex(5)

Found 874272 unique tokens.
बताए: 1
काम: 2
गरिएको: 3
नेपाल: 4
हजार: 5


In [199]:
# Prepare text sequences
sequences = tokenizer.texts_to_sequences(newsTexts)

# helps to shortens the sequence to specified max length
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# convert list of class labels to a binary class matrix
labelsCategorical = to_categorical(np.asarray(labels))

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labelsCategorical.shape)
print("\n")
print('First Label (Numpy Array):', labelsCategorical[0])
print("\n")
print('Second Label (Numpy Array):', labelsCategorical[1])

Shape of data tensor: (104047, 1000)
Shape of label tensor: (104047, 63)


First Label (Numpy Array): [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Second Label (Numpy Array): [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [200]:
indices = np.arange(data.shape[0])
print(indices)

#Shuffle dataset into random order
np.random.shuffle(indices)
data = data[indices]
finalLabels = labelsCategorical[indices]

# split the data into a training set(90%) and a validation set(10%)
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

# train news text and category set
x_train = data[:-num_validation_samples]
y_train = finalLabels[:-num_validation_samples]

# test news text and category set
x_val = data[-num_validation_samples:]
y_val = finalLabels[-num_validation_samples:]

print("X train shape: ", x_train.shape)
print("Y train shape: ", y_train.shape)
print("X test shape: ", x_val.shape)
print("Y test shape: ", y_val.shape)

[     0      1      2 ... 104044 104045 104046]
X train shape:  (93643, 1000)
Y train shape:  (93643, 63)
X test shape:  (10404, 1000)
Y test shape:  (10404, 63)


In [ ]:
# # load json and create model
# # from keras.models import model_from_json 
# from tensorflow.keras.models import Sequential, model_from_json
# from keras_self_attention import SeqSelfAttention


# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json, custom_objects={'SeqSelfAttention': SeqSelfAttention})
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")
 
# # evaluate loaded model on test data
# # loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# loaded_model.compile(optimizer='adam', loss='mse', metrics=['acc', f1_m, precision_m, recall_m])

# score = loaded_model.evaluate(x_val, y_val, verbose=0)
# index = 0
# for metric in loaded_model.metrics_names:
#     print("%s: %.2f%%" % (metric, score[index]*100))
#     index = index + 1

In [201]:
print("Preparing Embedding Layer")
# Get maximum number of words 
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)

# Creates numpy array of shape (num_words, EMBEDDING_DIM)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

# Map each word token to vector using pre-trained embedding dictionary
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
print("Done")

Preparing Embedding Layer
Done


In [ ]:
# print('Training model. CNN CNN Model.')

# # train a 1D convnet with global maxpooling
# sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
# embedded_sequences = embedding_layer(sequence_input)

# x = Conv1D(128, 5, activation='relu')(embedded_sequences)
# x1 = Dropout(0.2)(x)
# x2 = MaxPooling1D(5)(x1)
# x3 = Conv1D(128, 5, activation='relu')(x2)
# x4 = Dropout(0.3)(x3)
# x5 = MaxPooling1D(5)(x4)
# x6 = Conv1D(128, 5, activation='relu')(x5)
# x7 = GlobalMaxPooling1D()(x6)
# x8 = Dense(128, activation='relu')(x7)
# print("len labels:", str(len(labels_index)))
# preds = Dense(len(labels_index), activation='softmax')(x8)

# print("Preds shape: ", preds.shape)
# print("sequence_input shape: ", sequence_input.shape)

# model = Model(sequence_input, preds)
# model.compile(loss='categorical_crossentropy',
#               optimizer='rmsprop',
#               metrics=['acc', f1_m, precision_m, recall_m])

# print("Shape of x_train: ", x_train.shape)
# print("Shape of y_train: ", y_train.shape)
# print("Shape of x_val: ", x_val.shape)
# print("Shape of y_val: ", y_val.shape)
# print(model.output_shape)
# model.summary()


# history = model.fit(x_train, y_train,
#           batch_size=128,
#           epochs=15,
#           validation_data=(x_val, y_val))

In [ ]:
# print('Training model. CNN biLstm Model.')
# from keras.models import Sequential
# from keras.layers import Embedding
# from tensorflow.keras.layers import Bidirectional
# from keras.layers import LSTM
# from keras.layers import Activation
# from keras.layers import Flatten


# sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
# embedded_sequences = embedding_layer(sequence_input)

# model = Sequential()
# model.add(embedding_layer)
# model.add(Conv1D(128, 5, activation='relu'))
# model.add(MaxPooling1D(5))
# model.add(Dropout(0.2)) # embedding dropouts
# model.add(Bidirectional(LSTM(256, return_sequences=True, recurrent_dropout=0.2, activation = 'tanh'))) # weight drop on recurrent layers using recurrent_dropout
# # model.add(GlobalMaxPooling1D())
# # model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dense(63, activation="softmax"))

# model.summary()
# # model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam', metrics = ['accuracy'])
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc', f1_m, precision_m, recall_m])


# train the model
# model.fit(x_train, y_train, X_val=x_val, y_val=y_val,
#     epoch_num=15, optimizer='adam', verbose=True)
# history = model.fit(x_train, y_train,
#           batch_size=128,
#           epochs=15,
#           validation_data=(x_val, y_val))

In [202]:
# Hybrid Model: CNN Bilstm attention Model 
def hybrid_model():
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    x = Conv1D(filters = 128, kernel_size = 5, activation = 'relu')(embedded_sequences)  #, padding = 'same'
    x = MaxPooling1D(5)(x)
    x = Dropout(0.2)(x)
    lstm_out = Bidirectional(LSTM(128, return_sequences=True))(x)
    lstm_out = Dropout(0.2)(lstm_out)
    attention_mul = SeqSelfAttention(attention_width=15, attention_activation='relu',name='Attention')(lstm_out)
    attention_mul = Flatten()(attention_mul)
    output = Dense(63, activation='softmax')(attention_mul)
    model = Model(inputs=[sequence_input], outputs=output)
    return model

# Model: Attention Model 
def attention_model():
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    attention_mul = SeqSelfAttention(attention_width=15, attention_activation='relu',name='Attention')(embedded_sequences)
    attention_mul = Dropout(0.2)(attention_mul)
    attention_mul = Flatten()(attention_mul)
    output = Dense(63, activation='sigmoid')(attention_mul)
    model = Model(inputs=[sequence_input], outputs=output)
    return model

# Model: Bi-LSTM Model 
def bilstm_model():
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    x = Bidirectional(LSTM(128, return_sequences=True))(embedded_sequences)
    x1 = Dropout(0.2)(x)
    x2 = Flatten()(x1)
    output = Dense(63, activation='softmax')(x2)
    model = Model(inputs=[sequence_input], outputs=output)
    return model

# Model: CNN Model 
def cnn_model():
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    x = Conv1D(filters = 128, kernel_size = 5, activation = 'relu')(embedded_sequences) 
    x1 = MaxPooling1D(5)(x)
    x2 = Dropout(0.2)(x1)
    x3 = Flatten()(x2)
    output = Dense(63, activation='softmax')(x3)
    model = Model(inputs=[sequence_input], outputs=output)
    return model

m = hybrid_model()
m.summary()


m.compile(optimizer='adam', loss='mse', metrics=['acc', f1_m, precision_m, recall_m])

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 1000)]            0         
                                                                 
 embedding_10 (Embedding)    (None, 1000, 300)         6000000   
                                                                 
 conv1d_14 (Conv1D)          (None, 996, 128)          192128    
                                                                 
 max_pooling1d_14 (MaxPoolin  (None, 199, 128)         0         
 g1D)                                                            
                                                                 
 dropout_30 (Dropout)        (None, 199, 128)          0         
                                                                 
 bidirectional_16 (Bidirecti  (None, 199, 256)         263168    
 onal)                                                    

In [203]:
history = m.fit([x_train], y_train, epochs=100, batch_size=64, validation_split=0.1, validation_data=(x_val, y_val))

1464/1464 [==============================] - 967s 659ms/step - loss: 0.0074 - acc: 0.6773 - f1_m: 0.6701 - precision_m: 0.7886 - recall_m: 0.5862 - val_loss: 0.0065 - val_acc: 0.7234 - val_f1_m: 0.7121 - val_precision_m: 0.8151 - val_recall_m: 0.6333


In [ ]:
# BiLstm CNN Model 
# from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
# from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
# from keras.optimizers import Adam

# sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH, ))
# x = Embedding(num_words, EMBEDDING_DIM, weights=[embedding_matrix],trainable = False)(sequence_input)
# x = SpatialDropout1D(0.2)(x)
# x = Bidirectional(LSTM(128, return_sequenc `1es=True,dropout=0.1,recurrent_dropout=0.1))(x)
# x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
# avg_pool = GlobalAveragePooling1D()(x)
# max_pool = GlobalMaxPooling1D()(x)
# x = concatenate([avg_pool, max_pool]) 
# # x = Dense(128, activation='relu')(x)
# # x = Dropout(0.1)(x)
# preds = Dense(63, activation="sigmoid")(x)
# model = Model(sequence_input, preds)
# model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-3),metrics=['accuracy'])
# model.summary()
# history = model.fit(x_train, y_train, epochs = 20, validation_split=0.1, shuffle=True, batch_size=64)

In [ ]:
# CNN Bilstm attention
# from keras.layers import Input, Dense, LSTM, concatenate,Conv1D,Dropout,Bidirectional,Multiply
# from keras.models import Model
# from keras_self_attention import SeqSelfAttention

# from tensorflow.keras.layers import Attention
# from keras.layers.core import *
# from keras.models import *

# lstm_units = 64
# SINGLE_ATTENTION_VECTOR = False

# def bilstm_model():
#     sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
#     embedded_sequences = embedding_layer(sequence_input)
#     x = Bidirectional(LSTM(128, return_sequences=True))(embedded_sequences)
#     x1 = Dropout(0.2)(x)
#     output = Dense(63, activation='sigmoid')(x1)
#     model = Model(inputs=[sequence_input], outputs=output)
#     return model
# modelsdf = bilstm_model()
# modelsdf.summary()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import pyplot


# plot metrics
pyplot.plot(history.history["f1_m"],  label="f1 score")

pyplot.plot(history.history["recall_m"], label="recall")
pyplot.plot(history.history['acc'] , label="accuracy")
pyplot.title('Evaluation stats (Train)')
pyplot.xlabel("epoch")
pyplot.legend(loc="upper left")
pyplot.show()

In [ ]:
pyplot.plot(history.history["val_f1_m"],  label="Val f1 score")

pyplot.plot(history.history["val_recall_m"], label="Val recall")
pyplot.plot(history.history['val_acc'] , label="Val accuracy")
pyplot.xlabel("epoch")

pyplot.title('Evaluation stats (Test)')
pyplot.legend(loc="upper left")
pyplot.show()

In [ ]:
# plot metrics
import matplotlib.pyplot as pyplot

pyplot.plot(history.history["loss"],  label="loss (train)")
pyplot.xlabel("epoch")
pyplot.ylabel("loss")

pyplot.title('Model Loss (train)')
pyplot.legend(loc="upper left")
pyplot.show()

In [ ]:
# plot metrics
import matplotlib.pyplot as pyplot

pyplot.plot(history.history["val_loss"],  label="loss (test)")
pyplot.xlabel("epoch")
pyplot.ylabel("loss")

pyplot.title('Model Loss (test)')
pyplot.legend(loc="upper left")
pyplot.show()

In [ ]:
# serialize model to JSON
model_json = m.to_json()
with open("proposed_hybrid_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
m.save_weights("proposed_hybrid_model.h5")
print("Saved model to disk")

In [ ]:
from gensim.models.keyedvectors import KeyedVectors

print("*****Loading pre-trained GloVe model for Nepali*******")
# Load the pre-trained GloVe model for Nepali
wv = KeyedVectors.load_word2vec_format('processed.glove', binary=False)
print("*****Completed*******")


In [ ]:
# Classify News Text to Category (Step 1)

politicsNews = "नेपाली काँग्रेसका सांसद बद्रीप्रसाद पाण्डेले एमाले अध्यक्ष केपी शर्मा ओलीले सरकारमाथि नै धावा बोल्ने काम गरिरहेको आरोप लगाउनु भएको छ । शनिबार प्रतिनिधिसभा बैठकमा बोल्दै उहाँले संविधानले नचिनेको ‘उच्चस्तरीय राजनीतिक संयन्त्रको संयोजक हुँ’ भन्दै ओलीले कार्यपालिकामाथि नै धावा बोल्ने काम गरिरहेको बताउनु भएको हो । सांसद पाण्डेले प्रतिनिधिसभा विघटन गर्नु संविधान विपरित हो भन्ने अदालतको फैसलामाथि नै अपमान गर्ने काम ओलीले गरिरहेको भन्दै आक्रोश समेत व्यक्त गर्नु भयो । संविधानले प्रधानमन्त्रीभन्दा माथि कार्यपालिकामा कोही पनि नहुने व्यवस्था गरे पनि ओलीले आफूलाई माथि छु भन्ने ठानेको उहाँको भनाई थियो ।"
chinaNews = "चीनमा गत जनवरी १३ देखि १९ को बीचमा लगभग १३ हजार कोभिड– १९ सङ्क्रमणको कारण मृत्यु भएको छ । गत जनवरी १२ सम्म अस्पतालमा कोभिडका कारण ६० हजार मानिसको मृत्यु भइसकेको थियो । अस्पतालमा भर्ना भएका छ सय ८१ बिरामीको कोरोना भाइरस सङ्क्रमणका कारण श्वासप्रश्वासको समस्याबाट मृत्यु भएको र एघार हजार नौ सय ७७ जनाको सङ्क्रमणसँगै अन्य रोगका कारण मृत्यु भएका चीनको सेन्टर फर डिजिज कन्ट्रोल एण्ड प्रिभेन्सन (सिडिसि) ले शनिबार जारी एक विज्ञप्तिमा जनाएको छ । गत डिसेम्बरमा चीनले शून्य–कोभिड नीति त्यागेपछि यस रोगबाट ६० हजार भन्दा बढी मानिसहरूको मृत्यु भएको सिडिसिले जनाएको छ ।"
sportsNews = "इटहरी उपमहानगरपालिकास्थित जनता माविको खेलमैदानमा आज सम्पन्न प्रधानमन्त्री कप महिला राष्ट्रिय टी-२० क्रिकेट प्रतियोगिताको उपाधि सुदूरपश्चिम प्रदेशले जितेको छ । फाइनल खेलमा घरेलु टोली प्रदेश नं १ लाई छ विकेटले पराजित गर्दै सुदूरपश्चिम विजयी भएको हो । प्रदेश नं १ ले दिएको एक सय दुई रनको लक्ष्य पाएको सुदूरपश्चिमले एक बल बाँकी छँदा लक्ष्य पूरा गरेको थियो । सुदूरपश्चिमको जितका लागि अन्तिम ओभरमा आठ रन आवश्यक आवश्यक थियो । जसमा ओभरको दोस्रो बलमा कविता कुँवरले छक्का प्रहार गरी टीमलाई विजयी बनाइन् । खेलको सुरुआतमा राम्रो प्रदर्शन नसकेको सुदूरपश्चिमको खेल मध्यक्रमका ब्याटर सम्झना खड्का र डली भट्टले सम्हालेका थिए । सम्झनाले ३३ बलमा दुई चौकासहित अविजित २७ रन बनाउँदा डलीले १६ बलमा एक चौका र दुई छक्कासहित २२ रनको योगदान दिएकी थिइन् । कविता १९ रनमा अविजित भइन् । बलिङतर्फ सवनम राई, हिरनमई रोय, रुविना क्षेत्री र सङ्गीता राईले १-१ विकेट लिएका थिए । टस जितेर पहिले ब्याटिङ गरेको प्रदेश नं १ ले २० ओभरमा पाँच विकेटको क्षतिमा एक सय एक रन बनाएको थियो । प्रदेश नं १ का लागि ओपनर काजल श्रेष्ठ र कप्तान रुविनाले ३६ रनको सुरुआती साझेदारी गरेका थिए ।"
economicNews = "नेपाल ट्रक यातायात व्यवसायी महासंघको एक प्रतिनिधि मण्डलले उपप्रधान तथा अर्थमन्त्री विष्णुप्रसाद पौडेलबीच शुक्रबार अर्थ मन्त्रालयमा भेटघाट सम्पन्न भएको छ । हाल यातायात क्षेत्रको ढुवानीमा देखिएको समस्याको बारेमा उक्त प्रतिनिधि मण्डलले अर्थमन्त्रीको ध्यानाकर्षण गराएको छ । भेटका क्रममा मालबाहक ढुवानी क्षेत्रमा देखिएका समस्याका बारेमा पनि अर्थ मन्त्रीलाई जानकारी गराइएको थियो। अर्थमन्त्री पौडेलले व्यवसायीको समस्या समाधानका पहल गर्ने प्रतिबद्धता व्यक्त गरेको महासंघका अध्यक्ष हिरालाल श्रेष्ठले जानकारी दिए । भेटघाट कार्यक्रममा महासंघका अध्यक्ष हिरालाल श्रेष्ठ, महासचिव राजेन्द्र विक्रम बानियाँ , कोषाध्यक्ष ओम कार्की लागतका व्यवसायीहरूको सहभागिता रहेको थियो । प्रतिनिधिमण्डलले मन्त्री पौडेललाई सफल कार्यकालका लागि शुभकामना पनि व्यक्त गरेको छ ।"
testNews = [politicsNews,
           sportsNews,
           economicNews,
           chinaNews,
          ]

# Tokenize the input text data for prediction
tokenizer1 = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer1.fit_on_texts(testNews)
sequences1 = tokenizer.texts_to_sequences(testNews)
testData = pad_sequences(sequences1, maxlen=MAX_SEQUENCE_LENGTH)

print('Shape of data tensor:', testData.shape)


In [ ]:
# Classify News Text to Category (Step 2)

def print_prediction(testData, model):
    print("*****EVALUATE REVIEW*******")
    y_pred = model.predict(testData)
    # Find the index of the highest probability for each sample
    predicted_categories = y_pred.argmax(axis=1)
    for each_review, each_y_pred in zip(reviews, predicted_categories):
        print("News Text:\n{0}\nCategory -> {1}".format(each_review, get_key(each_y_pred, labels_index)))
        print("\n")

# Gets CategoryName from label index
def get_key(value, my_dict):
    for key, val in my_dict.items():
        if val == value:
            return key
    return None
print_prediction(testData, m)

In [ ]:
# load json and create model
# from keras.models import model_from_json 
from tensorflow.keras.models import Sequential, model_from_json
from keras_self_attention import SeqSelfAttention


json_file = open('proposed_hybrid_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json, custom_objects={'SeqSelfAttention': SeqSelfAttention})
# load weights into new model
loaded_model.load_weights("proposed_hybrid_model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
score = loaded_model.evaluate(x_val, y_val, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))